In [2]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [3]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
import LinearAlgebra: ldiv!, dot
using Test
using ForwardDiff
using SpecialFunctions
using BenchmarkTools
using TransportMap
using QuadGK
using AdaptiveTransportMap: transform!, evaluate, ncoeff, optimize, negative_log_likelihood!
using DiffResults
using Distributions
using Random
using LoopVectorization
using Optim
using NLsolve
using Polynomials
using Optim: InverseDiagonal
using MLDataUtils
using Test

### Greedy fit

In [104]:
Nx = 2
Ne = 8
m = 10
X = Matrix([0.267333   1.43021;
              0.364979   0.607224;
             -1.23693    0.249277;
             -2.0526     0.915629;
             -0.182465   0.415874;
              0.412907   1.01672;
              1.41332   -0.918205;
              0.766647  -1.00445]')

B = MultiBasis(CstProHermite(6; scaled =true), Nx)

idx = [0 1; 0 2]
coeff = [0.0; 0.0]

f = ExpandedFunction(B, idx, coeff)
fp = ParametricFunction(f)
R = IntegratedFunction(fp)

C = MapComponent(R)

MapComponent(8, 2, 2, IntegratedFunction(8, 2, 2, Rectifier("softplus"), ParametricFunction(ExpandedFunction(8, 2, 2, MultiBasis(Basis of 8 functions:
ProPolyHermite(0, ImmutablePolynomial(1.0), false)
ProHermite(0, ProPolyHermite(0, ImmutablePolynomial(0.6316187777460647), true), true)
ProHermite(1, ProPolyHermite(1, ImmutablePolynomial(0.6316187777460647*x), true), true)
ProHermite(2, ProPolyHermite(2, ImmutablePolynomial(-0.44662192086900115 + 0.44662192086900115*x^2), true), true)
ProHermite(3, ProPolyHermite(3, ImmutablePolynomial(-0.7735718587191167*x + 0.25785728623970555*x^3), true), true)
ProHermite(4, ProPolyHermite(4, ImmutablePolynomial(0.38678592935955836 - 0.7735718587191167*x^2 + 0.12892864311985278*x^4), true), true)
ProHermite(5, ProPolyHermite(5, ImmutablePolynomial(0.8648796307884041*x - 0.5765864205256028*x^3 + 0.05765864205256028*x^5), true), true)
ProHermite(6, ProPolyHermite(6, ImmutablePolynomial(-0.353085630726383 + 1.059256892179149*x^2 - 0.353085630726383*x^4

In [ ]:
optimize(C, X, 3)

In [105]:
optimize(C, X, nothing)

(MapComponent(8, 2, 2, IntegratedFunction(8, 2, 2, Rectifier("softplus"), ParametricFunction(ExpandedFunction(8, 2, 2, MultiBasis(Basis of 8 functions:
ProPolyHermite(0, ImmutablePolynomial(1.0), false)
ProHermite(0, ProPolyHermite(0, ImmutablePolynomial(0.6316187777460647), true), true)
ProHermite(1, ProPolyHermite(1, ImmutablePolynomial(0.6316187777460647*x), true), true)
ProHermite(2, ProPolyHermite(2, ImmutablePolynomial(-0.44662192086900115 + 0.44662192086900115*x^2), true), true)
ProHermite(3, ProPolyHermite(3, ImmutablePolynomial(-0.7735718587191167*x + 0.25785728623970555*x^3), true), true)
ProHermite(4, ProPolyHermite(4, ImmutablePolynomial(0.38678592935955836 - 0.7735718587191167*x^2 + 0.12892864311985278*x^4), true), true)
ProHermite(5, ProPolyHermite(5, ImmutablePolynomial(0.8648796307884041*x - 0.5765864205256028*x^3 + 0.05765864205256028*x^5), true), true)
ProHermite(6, ProPolyHermite(6, ImmutablePolynomial(-0.353085630726383 + 1.059256892179149*x^2 - 0.353085630726383*x^

In [106]:
greedyfit(m, Nx, X, 3)

1 terms - Training error: 1.3172777885451104
1 terms - Training error: 1.259805580256394
2 terms - Training error: 1.193580666838948
3 terms - Training error: 1.177493191564905


(MapComponent(10, 3, 2, IntegratedFunction(10, 3, 2, Rectifier("softplus"), ParametricFunction(ExpandedFunction(10, 3, 2, MultiBasis(Basis of 10 functions:
ProPolyHermite(0, ImmutablePolynomial(1.0), false)
ProHermite(0, ProPolyHermite(0, ImmutablePolynomial(0.6316187777460647), true), true)
ProHermite(1, ProPolyHermite(1, ImmutablePolynomial(0.6316187777460647*x), true), true)
ProHermite(2, ProPolyHermite(2, ImmutablePolynomial(-0.44662192086900115 + 0.44662192086900115*x^2), true), true)
ProHermite(3, ProPolyHermite(3, ImmutablePolynomial(-0.7735718587191167*x + 0.25785728623970555*x^3), true), true)
ProHermite(4, ProPolyHermite(4, ImmutablePolynomial(0.38678592935955836 - 0.7735718587191167*x^2 + 0.12892864311985278*x^4), true), true)
ProHermite(5, ProPolyHermite(5, ImmutablePolynomial(0.8648796307884041*x - 0.5765864205256028*x^3 + 0.05765864205256028*x^5), true), true)
ProHermite(6, ProPolyHermite(6, ImmutablePolynomial(-0.353085630726383 + 1.059256892179149*x^2 - 0.35308563072638